# Prepare data for analysis

In [303]:
import pandas as pd
from collections import Counter
# Specify the filename of the raw Daylio export
filename = 'daylio_export_2023_10_07.csv'
# Specify the project name
project_name = 'oct_7_pilot'
# Specify the removal threshold
freq_threshold = 3

### Read app data

In [304]:
# Read CSV file exported from the Daylio app
df = pd.read_csv(filename)
# Drop diary note columns
df.drop(columns=['note_title', 'note'], inplace=True)
df.head()

,full_date,date,weekday,time,mood,activities
0,2023-10-07,October 7,Saturday,18:35,good,kitchen | good sleep | mother | family | home ...
1,2023-10-06,October 6,Friday,21:03,bad,tired | kitchen | capuchino | fasting | bad sl...
2,2023-10-05,October 5,Thursday,23:59,meh,tired | kitchen | capuchino | fasting | medium...
3,2023-10-04,October 4,Wednesday,22:14,good,grateful | happy | kitchen | restaurant | bad ...
4,2023-10-03,October 3,Tuesday,20:28,bad,tired | stressed | fasting | bad sleep | homew...


### Create datetime column

In [305]:
# Combine the date and time columns into a single one
df['datetime'] = pd.to_datetime(df['full_date'] + ' ' + df['time'], format='%Y-%m-%d %H:%M')
# Print earliest and latest dates
print(f"From {df['datetime'].min()} to {df['datetime'].max()}")

From 2023-08-03 12:55:00 to 2023-10-07 18:35:00


### Scale ordinal mood labels

In [306]:
# Map each mood label to ordinal scale 
mood_scale = {
    'rad': 1.00,
    'good': 0.75,
    'meh': 0.50,
    'bad': 0.25,
    'awful': 0.00 
}
# Create the mood target column
df['mood'] = df['mood'].map(mood_scale)
df.head()

,full_date,date,weekday,time,mood,activities,datetime
0,2023-10-07,October 7,Saturday,18:35,0.75,kitchen | good sleep | mother | family | home ...,2023-10-07 18:35:00
1,2023-10-06,October 6,Friday,21:03,0.25,tired | kitchen | capuchino | fasting | bad sl...,2023-10-06 21:03:00
2,2023-10-05,October 5,Thursday,23:59,0.50,tired | kitchen | capuchino | fasting | medium...,2023-10-05 23:59:00
3,2023-10-04,October 4,Wednesday,22:14,0.75,grateful | happy | kitchen | restaurant | bad ...,2023-10-04 22:14:00
4,2023-10-03,October 3,Tuesday,20:28,0.25,tired | stressed | fasting | bad sleep | homew...,2023-10-03 20:28:00


### Add the following record's mood

In [307]:
# Sort record by datetime ascending
df.sort_values(by='datetime', ascending=True, inplace=True)
# Put next day mood using shift
df['next_day_mood'] = df['mood'].shift(-1)

### Scale ordinal sleep quality labels

In [308]:
# Map each sleep quality label to ordinal scale 
sleep_scale = {
    'sleep early': 1.00,
    'good sleep': 1.00,
    'medium sleep': 0.60,
    'bad sleep': 0.00,
    'hangover': 0.45
}
# Parse activities into list
df['activities'] = df['activities'].apply(lambda x: x.split(' | '))
# Select sleep activities 
df['sleep'] = df['activities'].apply(lambda x: [i for i in x if i in sleep_scale.keys()])
# Remove sleep activities labels
df['activities'] = df['activities'].apply(lambda x: [i for i in x if i not in sleep_scale.keys()])
# Create the sleep quality target column
df['sleep'] = df['sleep'].apply(lambda x: sleep_scale[x[0]] if len(x)>0 else None)
# Mask null values with mean average
mean_sleep_quality = df['sleep'].mean()
df['sleep'].fillna(mean_sleep_quality, inplace=True)
df.head()


,full_date,date,weekday,time,mood,activities,datetime,next_day_mood,sleep
54,2023-08-03,August 3,Thursday,12:55,0.25,"[sad, movies, gaming]",2023-08-03 12:55:00,0.75,1.000000
53,2023-08-03,August 3,Thursday,14:36,0.75,[Reels],2023-08-03 14:36:00,1.00,0.634694
52,2023-08-03,August 3,Thursday,20:02,1.00,[concert],2023-08-03 20:02:00,0.25,0.634694
51,2023-08-04,August 4,Friday,20:17,0.25,"[sad, restaurant, mother, father]",2023-08-04 20:17:00,0.50,0.450000
50,2023-08-05,August 5,Saturday,20:34,0.50,"[stressed, kitchen, mother, father, Priežu Vēj...",2023-08-05 20:34:00,0.75,1.000000


### Scale fasting labels

In [309]:
def scale_fasting(activities):
    """Scale fasting based on cappuchino and fasting labels."""
    # Set 1.0 or 0.0 if fasted
    result = 1 if 'fasting' in activities else 0
    # Deduct 0.2 if I drank coffee when fasted
    if result == 1 and 'capuchino' in activities:
        result -= 0.2
    return result
# Scale fasting activities
df['fasting'] = df['activities'].apply(scale_fasting)
# Remove fasting activities labels
fasting_labels = ['fasting', 'capuchino']
df['activities'] = df['activities'].apply(lambda x: [i for i in x if i not in fasting_labels])
df.head()

,full_date,date,weekday,time,mood,activities,datetime,next_day_mood,sleep,fasting
54,2023-08-03,August 3,Thursday,12:55,0.25,"[sad, movies, gaming]",2023-08-03 12:55:00,0.75,1.000000,0.0
53,2023-08-03,August 3,Thursday,14:36,0.75,[Reels],2023-08-03 14:36:00,1.00,0.634694,0.0
52,2023-08-03,August 3,Thursday,20:02,1.00,[concert],2023-08-03 20:02:00,0.25,0.634694,0.0
51,2023-08-04,August 4,Friday,20:17,0.25,"[sad, restaurant, mother, father]",2023-08-04 20:17:00,0.50,0.450000,0.0
50,2023-08-05,August 5,Saturday,20:34,0.50,"[stressed, kitchen, mother, father, Priežu Vēj...",2023-08-05 20:34:00,0.75,1.000000,0.0


### Scale university labels

In [310]:
# Map each university label to engagement scale 
university_to_engagement = {
    'lectures': 0.50,
    'study': 0.70,
    'homework': 1.00,
    'exam': 1.00,
    'group project': 0.80
}
university_labels = university_to_engagement.keys()
# Method to scale university activity
def scale_university(activities):
    """Scale university based on engagement."""
    # Select day's university engagement
    university_engagement = [university_to_engagement[i] for i in activities if i in university_labels]
    # Calculate sum of all university engagement
    if university_engagement:
        total_engagement = sum(university_engagement)
        if total_engagement > 1:
            return 1
        return total_engagement
    # Return 0 if no university engagement
    return 0
# Scale university activities
df['university'] = df['activities'].apply(scale_university)
# Remove university activities labels
df['activities'] = df['activities'].apply(lambda x: [i for i in x if i not in university_labels])
df.head()

,full_date,date,weekday,time,mood,activities,datetime,next_day_mood,sleep,fasting,university
54,2023-08-03,August 3,Thursday,12:55,0.25,"[sad, movies, gaming]",2023-08-03 12:55:00,0.75,1.000000,0.0,0.0
53,2023-08-03,August 3,Thursday,14:36,0.75,[Reels],2023-08-03 14:36:00,1.00,0.634694,0.0,0.0
52,2023-08-03,August 3,Thursday,20:02,1.00,[concert],2023-08-03 20:02:00,0.25,0.634694,0.0,0.0
51,2023-08-04,August 4,Friday,20:17,0.25,"[sad, restaurant, mother, father]",2023-08-04 20:17:00,0.50,0.450000,0.0,0.0
50,2023-08-05,August 5,Saturday,20:34,0.50,"[stressed, kitchen, mother, father, Priežu Vēj...",2023-08-05 20:34:00,0.75,1.000000,0.0,0.0


### Scale job labels

In [311]:
# Map each job labels to scale 
job_scale = {
    'bare minimum': 0.50,
    'good work': 0.80,
    'extras': 0.90,
    'overtime': 1.00,
}
# Select job activities 
df['job'] = df['activities'].apply(lambda x: [i for i in x if i in job_scale.keys()])
# Remove job activities labels
df['activities'] = df['activities'].apply(lambda x: [i for i in x if i not in job_scale.keys()])
# Create the job target column
df['job'] = df['job'].apply(lambda x: job_scale[x[0]] if len(x)>0 else 0)
df.head()

,full_date,date,weekday,time,mood,activities,datetime,next_day_mood,sleep,fasting,university,job
54,2023-08-03,August 3,Thursday,12:55,0.25,"[sad, movies, gaming]",2023-08-03 12:55:00,0.75,1.000000,0.0,0.0,0.0
53,2023-08-03,August 3,Thursday,14:36,0.75,[Reels],2023-08-03 14:36:00,1.00,0.634694,0.0,0.0,0.0
52,2023-08-03,August 3,Thursday,20:02,1.00,[concert],2023-08-03 20:02:00,0.25,0.634694,0.0,0.0,0.0
51,2023-08-04,August 4,Friday,20:17,0.25,"[sad, restaurant, mother, father]",2023-08-04 20:17:00,0.50,0.450000,0.0,0.0,0.0
50,2023-08-05,August 5,Saturday,20:34,0.50,"[stressed, kitchen, mother, father, Priežu Vēj...",2023-08-05 20:34:00,0.75,1.000000,0.0,0.0,0.0


### Remove sick days and hangovers

In [312]:
# List of sicknesses
sicknesses = ['September 22', 'September 24', 'September 25', 'September 26'] 
# Remove university activities labels
df = df[~df['date'].isin(sicknesses)]
df.head()

,full_date,date,weekday,time,mood,activities,datetime,next_day_mood,sleep,fasting,university,job
54,2023-08-03,August 3,Thursday,12:55,0.25,"[sad, movies, gaming]",2023-08-03 12:55:00,0.75,1.000000,0.0,0.0,0.0
53,2023-08-03,August 3,Thursday,14:36,0.75,[Reels],2023-08-03 14:36:00,1.00,0.634694,0.0,0.0,0.0
52,2023-08-03,August 3,Thursday,20:02,1.00,[concert],2023-08-03 20:02:00,0.25,0.634694,0.0,0.0,0.0
51,2023-08-04,August 4,Friday,20:17,0.25,"[sad, restaurant, mother, father]",2023-08-04 20:17:00,0.50,0.450000,0.0,0.0,0.0
50,2023-08-05,August 5,Saturday,20:34,0.50,"[stressed, kitchen, mother, father, Priežu Vēj...",2023-08-05 20:34:00,0.75,1.000000,0.0,0.0,0.0


### One-hot encode other activities

In [313]:
# Count all unique activity occurences
activity_labels = [label for activities in df['activities'] for label in activities]
label_to_count = dict(Counter(activity_labels))
# Remove all low frequency labels
high_freq_labels = [label for label in label_to_count.keys() if label_to_count[label] >= freq_threshold]
df['activities'] = df['activities'].apply(lambda x: [i for i in x if i in high_freq_labels])
# Create columns for one-hot encoding
for label in high_freq_labels:
    df[label] = 0
# Fill one-hot encodings
for i, row in df.iterrows():
    for label in row['activities']:
        df.at[i, label] = 1
# Drop activity column
df.drop(columns=['activities'], inplace=True)
df.head()

,full_date,date,weekday,time,mood,datetime,next_day_mood,sleep,fasting,university,...,sick,procrastinating,relax,snacking,office,W,YouTube Education,L,coffee overdose,unlucky
54,2023-08-03,August 3,Thursday,12:55,0.25,2023-08-03 12:55:00,0.75,1.000000,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
53,2023-08-03,August 3,Thursday,14:36,0.75,2023-08-03 14:36:00,1.00,0.634694,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
52,2023-08-03,August 3,Thursday,20:02,1.00,2023-08-03 20:02:00,0.25,0.634694,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
51,2023-08-04,August 4,Friday,20:17,0.25,2023-08-04 20:17:00,0.50,0.450000,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
50,2023-08-05,August 5,Saturday,20:34,0.50,2023-08-05 20:34:00,0.75,1.000000,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


### Save for Machine Learning

In [314]:
# Export timeseries dataset to CSV
df.to_csv(f'time_series_{project_name}.csv', index=False)
# Drop untrainable columns
df.drop(columns=['weekday', 'datetime', 'date', 'full_date', 'time'], inplace=True)
# Export machine learning dataset to CSV
df.to_csv(f'ml_{project_name}.csv', index=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50 entries, 54 to 0
Data columns (total 42 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   mood               50 non-null     float64
 1   next_day_mood      49 non-null     float64
 2   sleep              50 non-null     float64
 3   fasting            50 non-null     float64
 4   university         50 non-null     float64
 5   job                50 non-null     float64
 6   sad                50 non-null     int64  
 7   movies             50 non-null     int64  
 8   gaming             50 non-null     int64  
 9   Reels              50 non-null     int64  
 10  restaurant         50 non-null     int64  
 11  mother             50 non-null     int64  
 12  father             50 non-null     int64  
 13  stressed           50 non-null     int64  
 14  kitchen            50 non-null     int64  
 15  Priežu Vēji        50 non-null     int64  
 16  hydrated           50 non-null   